# HomeWord 1 : Linear Regression
本次实验的目标： 由前9个小时的18个特征预测第10个小时的PM2.5
[参考](https://colab.research.google.com/drive/131sSqmrmWXfjFZ3jWSELl8cm0Ox5ah3C#scrollTo=NzvXP5Jya64j)

In [27]:
import sys
import pandas as pd
import numpy as np

## Preprocssing 数据预处理
取需要的数值部分，将'RAINFALL'全部设置为0

In [ ]:
data = pd.read_csv('./train.csv', encoding='big5')

In [ ]:
data.head()

In [ ]:
# 抛去数据的前三列不要了
data = data.iloc[:, 3:]
# 将为NR的数据变为 0
data[data == 'NR'] = 0
raw_data = data.to_numpy()

In [ ]:
raw_data[0]

## Extract Features 特征提取 - 1 
将原始的4320*18的资料依照每个月组成12个月 18*480的数据类型(12*18*480) 其中18是feature 480 是 20*24 得来的

In [9]:
month_data = {}
for month in range(12):
    sample = np.empty([18, 480]) # 创建一个空样本大小为18*480
    for day in range(20): # 一个月只取了20天
        # 从raw_data中取数据 将数据填充进 sample
        sample[:, day * 24 : (day + 1) * 24] = raw_data[18 * (20 * month + day) : 18 * (20 * month + day + 1), :]
    month_data[month] = sample

## Extract Features 特征提取 - 2
每个月有480小时，每9个小时为一个data，每个月会有471个data，故总资料数为471\*12笔，而每笔数据有9\*18的feature(一个小时18个feature)

对应的target则有471 * 12 个 （第10个小时的PM2.5）

In [14]:
# 定义输入输出的大小
x = np.empty([12 * 471, 18 * 9], dtype=float)
y = np.empty([12 * 471, 1], dtype = float)

for month in range(12):
    for day in range(20):
        for hour in range(24):
            # 防止今天的数据跑到明天去
            if day == 19 and hour > 14:
                continue
            # 输入的sample大小为9*18
            x[month * 471 + day * 24 + hour, :] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1,-1)
            y[month * 471 + day + hour, 0] = month_data[month][9, day * 24 + hour + 9]


## Normalize 归一化

In [21]:
mean_x = np.mean(x, axis=0)
std_x = np.std(x, axis= 0)
for i in range(len(x)):
    for j in range(len(x[0])):
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]

上面的数据处理部分已懵逼！ 需要再看看咋做的 并想为啥这样操作

## Split Training Data Into "train_set" and "validation_set"
生成训练用的数据集train_set和验证数据局validation_set



In [24]:
import math
# floor() 返回数字的下舍整数。 取百分之80做训练集 剩下的 百分之20做测试集
x_train_set = x[: math.floor(len(x) * 0.8), :]
y_train_set = y[: math.floor(len(y) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8):, :]
y_validation = y[math.floor(len(x) * 0.8):, :]

## Training